In [1]:
from sqlalchemy import URL, create_engine, text
import streamlit as st

In [2]:
connection_string = URL.create(
    'postgresql',
    username=st.secrets.connections.postgresql.username,
    password=st.secrets.connections.postgresql.password,
    host=st.secrets.connections.postgresql.host,
    database=st.secrets.connections.postgresql.database,
    query={'options':'endpoint=ep-sweet-cloud-a1zh9huf'}
)

In [3]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'}, pool_pre_ping=True, pool_recycle=3600)
conn = engine.connect()

In [6]:
from _crawler import FilterURL, get_all_links, get_content
from _summarizer import create_model, summarize

summarizer_model = create_model('gemini-1.0-pro')

In [7]:
news_sample = get_content('https://e.vnexpress.net/news/news/education/hcmc-private-university-annual-tuition-cost-nearly-14-000-4734709.html')
summarized_text_sample = summarize(summarizer_model, full_text=news_sample['text'])

In [8]:
summarized_text_sample = summarized_text_sample.replace("'", "''").replace('"', '')
news_sample['title'] = news_sample['title'].replace('"', '')

In [9]:
query_insert = "INSERT INTO news(url, title, date, author, category, summary) " + \
                f"VALUES('{news_sample['url']}', '{news_sample['title']}', '{news_sample['date']}', '{news_sample['author']}', '{news_sample['category']}', '{summarized_text_sample}')"

In [10]:
conn.execute(text(query_insert))

In [11]:
conn.commit()

In [12]:
output = conn.execute(text("SELECT * FROM news"))

In [13]:
rows = output.fetchall()
for row in rows:
    print(row)

(1, 'https://e.vnexpress.net/news/property/asking-prices-for-uk-homes-close-to-record-high-4737334.html', 'Asking prices for UK homes close to record high', datetime.date(2024, 4, 22), 'Reuters', 'property', '**Summary:**\\n\\n* UK house asking prices are near record highs, increasing by 1.7% annually.\\n* Monthly asking prices rose by 1.1%, slowing from t ... (290 characters truncated) ... hile growth is slower among first-time and second-time buyers.\\n* Experts caution against substantial price growth due to current market conditions.')
(3, 'https://e.vnexpress.net/news/news/education/hcmc-private-university-annual-tuition-cost-nearly-14-000-4734709.html', 'HCMC private university annual tuition cost nearly $14,000', datetime.date(2024, 4, 18), 'Duong Tam', 'news', '**Tuition Fees for Private Universities in Ho Chi Minh City**\n\nAnnual tuition fees for new students at private universities in Ho Chi Minh City (HC ... (599 characters truncated) ... onsidering proficiency tests and En

In [14]:
conn.commit()

In [10]:
conn.close()